In [279]:
import pandas as pd
import random as rd 
df = pd.read_csv('GSAF5.csv', encoding = 'iso8859-1')

In [280]:
df.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN


In [281]:
# There are two columns that are unnnamed and contains no clear deta. I decided to gt rid of them right away..
# also href and href formula seems to be duplicates. 
df = df.drop(['Unnamed: 22','Unnamed: 23', 'href'],  axis = 1)

In [282]:
#just to check if they are gone
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [283]:
#I want som information about the data and missing values.
print (df.isnull().sum())
total_rows = df['Case Number'].count()
print('total number of rows:' , total_rows)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
Case Number.1                0
Case Number.2                0
original order               0
dtype: int64
total number of rows: 5992


In [284]:
''' There are som columns with quite alot of missing data. They dont seem to import as well, i dicided to drop them
if more than half of the data in thoose columns is missing. '''

for i in df.columns.values:
    if (df[i].isnull().sum() / total_rows > 0.5):
        df = df.drop([i], axis = 1)

In [285]:
# The column for href formula has only one value missing. I decided to get rid of that row.
df = df[df['href formula'].notnull()]        

In [286]:
# I also want to drop columns that has at least three values missing. 
df = df.dropna(axis = 0, thresh=3)

In [287]:
 #there are some values missing from the age column, i decided to fill them with the value for mean age

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

In [288]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Age'] = df['Age'].round(0)

In [289]:
#The values that are missing for the "activity" and "name" columns that I simply want to replace with unknown . 


df['Activity'] = df['Activity'].fillna('Unknown activity')
df['Name'] = df['Name'].fillna('Unknown')
df['Investigator or Source'] = df['Investigator or Source'].fillna('Unknown')
df['Injury'] = df['Injury'].fillna('Unknown')

In [319]:
print (df.isnull().sum())

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                     0
Name                         0
Sex                          0
Age                          0
Injury                      27
Fatal (Y/N)                 19
Species                   2933
Investigator or Source       0
pdf                          0
href formula                 0
Case Number.1                0
Case Number.2                0
original order               0
dtype: int64


In [291]:
pd.value_counts(df['Fatal (Y/N)'])

N          4315
Y          1551
UNKNOWN      94
 N            8
F             1
#VALUE!       1
n             1
N             1
Name: Fatal (Y/N), dtype: int64

In [292]:
'''There seems to be som wrong values, I'll see if theres a way to fix this.
First I just want to clear this up a little bit to see more properly'''

df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace("['Y']", 'Y').replace("['N']", 'N').replace(" N", 'N').replace("n", 'N').replace("F", 'N').replace("#VALUE!", 'N')

In [293]:
# This is just to see if it looks any better...

pd.value_counts(df['Fatal (Y/N)'])

N          4326
Y          1551
UNKNOWN      94
N             1
Name: Fatal (Y/N), dtype: int64

In [294]:
''' I want to replace the UNKNOWN with random picked values for yes or no. But theres a lot mor no fatal casuailtes, 
so I want to have that ratio in consideration while randomly defining new values for the UNKNOWN ones. 
Here I calculate the ratios to use.'''

total = 4327+1551
yes_ratio = (1551/total)*100
no_ratio = (4315/total)*100
outcomes = ['Y', 'N']

In [295]:
# first i'd like to retrive those rows where the unknown data for fatal is. 
unknown_data_fatal = df[df['Fatal (Y/N)'].values == 'UNKNOWN']

In [296]:
# Row I replace the values with an random selection for Y/N with consideration to the calculated ratio. 
unknown_rows_index = unknown_data_fatal.index.tolist()

for i in unknown_rows_index:
    df['Fatal (Y/N)'].loc[i] = rd.choices(outcomes, weights = [yes_ratio, no_ratio])
    


In [297]:
# just to check if what I wanted to happen happened. 
pd.value_counts(df['Fatal (Y/N)'])

N      4326
Y      1551
[N]      73
[Y]      21
N         1
Name: Fatal (Y/N), dtype: int64

In [298]:
# looks good just need som polish
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace("[N]", 'N').replace("[Y]", 'Y')

I want to do the same thing for the 'Sex' column.

In [310]:
pd.value_counts(df['Sex '])
df['Sex '] = df['Sex '].replace("lli", 'M').replace(".", 'M').replace("N", 'M')



In [315]:
total_sex = 4838+585
men_ratio = (4838/total)*100
female_ratio = (585/total)*100
outcomes_sex = ['M', 'F']

In [314]:
unknown_data_sex = df[df['Sex '].isnull()]

In [318]:
null_rows = unknown_data_sex.index.tolist()

for i in null_rows:
    df['Sex '].loc[i] = rd.choices(outcomes_sex, weights = [men_ratio, female_ratio])

In [317]:
pd.value_counts(df['Sex '])

M      4837
F       585
[M]     505
[F]      62
M         2
Name: Sex , dtype: int64

In [329]:
'''There are some alot more countries missing from the data set while Location is not, we should be able to find out
some of the countries that is missing based on the location of accident'''

Country_check = df[df['Country'].isnull()]
print(Country_check[['Country', 'Location','Area']])

#There is some country values I can fill in right here 



     Country                                   Location  \
303      NaN                                        NaN   
2731     NaN                                        NaN   
3153     NaN                                        NaN   
3162     NaN                  Between St. Kitts & Nevis   
3163     NaN                                        NaN   
3170     NaN                                        NaN   
3174     NaN                                        NaN   
3200     NaN                                        NaN   
3379     NaN                             Florida Strait   
3435     NaN                                        NaN   
3792     NaN                                        NaN   
4005     NaN                                        NaN   
4040     NaN                                Geyser Bank   
4271     NaN                Between  Cuba  & Costa Rica   
4412     NaN                225 miles east of Hong Kong   
4473     NaN                                        NaN 